In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/30-days-of-ml/sample_submission.csv
/kaggle/input/30-days-of-ml/train.csv
/kaggle/input/30-days-of-ml/test.csv


# Bas1c
> predicting a continuous target based on a number of feature columns given in the data. All of the feature columns, cat0 - cat9 are categorical, and the feature columns cont0 - cont13 are continuous.

## Data loading

In [2]:
train = pd.read_csv('/kaggle/input/30-days-of-ml/train.csv', index_col=0)
test = pd.read_csv('/kaggle/input/30-days-of-ml/test.csv', index_col=0)
train.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
id,,,,,,,,,,,,,,,,,,,,,
1,B,B,B,C,B,B,A,E,C,N,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
2,B,B,A,A,B,D,A,F,A,O,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
3,A,A,A,C,B,D,A,D,A,F,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
4,B,B,A,C,B,D,A,E,C,K,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
6,A,A,A,C,B,D,A,E,A,N,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [3]:
#feature target
y = train['target']
features = train.drop(['target'], axis = 1)
features.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
id,,,,,,,,,,,,,,,,,,,,,
1,B,B,B,C,B,B,A,E,C,N,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
2,B,B,A,A,B,D,A,F,A,O,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
3,A,A,A,C,B,D,A,D,A,F,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
4,B,B,A,C,B,D,A,E,C,K,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
6,A,A,A,C,B,D,A,E,A,N,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


## Data Preparation

In [4]:
#using ordinal encoding and save our encoded features as new variables X and X_test (mapping labels into integers :uniqueness)
object_cols = [col for col in features.columns if 'cat' in col]

#ordinal encode cat cols
X = features.copy()
X_test = test.copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

X_test.head()



,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
id,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,4.0,4.0,8.0,...,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702
5,0.0,1.0,0.0,2.0,1.0,2.0,0.0,4.0,2.0,7.0,...,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940
15,1.0,0.0,0.0,0.0,1.0,1.0,0.0,4.0,3.0,10.0,...,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099
16,1.0,1.0,0.0,2.0,1.0,3.0,0.0,4.0,0.0,13.0,...,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372
17,1.0,1.0,0.0,2.0,1.0,2.0,0.0,4.0,2.0,5.0,...,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412


In [5]:
#break off validation set / training set
X_train, X_valid, y_train, y_valid = train_test_split(X,y,random_state=42)

## Train Model (Random Forest Algo)

In [6]:
model = RandomForestRegressor(random_state=1)

model.fit(X_train, y_train)
preds_valid = model.predict(X_valid)
print(mean_squared_error(y_valid, preds_valid, squared=False))

0.734702709454555


# XGBoosting

In [7]:
from xgboost import XGBRegressor

xg_model = XGBRegressor()
xg_model.fit(X_train, y_train)

xg_predictions = xg_model.predict(X_valid)
print("MSE: " + str(mean_squared_error(xg_predictions, y_valid)))

MSE: 0.5235460033841066


## Parameter tuning

In [8]:
xg_model_tune_model = XGBRegressor(n_estimators = 500, learning_rate = 0.05)
xg_model_tune_model.fit(X_train, y_train, early_stopping_rounds = 5, eval_set=[(X_valid, y_valid)], verbose = False)
xg_model_tune_predictions = xg_model_tune_model.predict(X_valid)
print("MSE: " + str(mean_squared_error(xg_model_tune_predictions, y_valid)))

MSE: 0.5188944948279395


In [9]:
#XGBoost model with parameters tuning gives the lowest MSE
best_predictions = xg_model_tune_model.predict(X_test)

output = pd.DataFrame({'Id': X_test.index,
                       'target': best_predictions})
output.to_csv('submission.csv', index=False)